In [3]:
# imports and link xml
import re
from math import radians, cos, sin, asin, sqrt
import folium
from IPython.display import display
import pandas as pd
import xml.etree.ElementTree as ET
tree = ET.parse('session/Kismet-20191011-18-31-55-1.netxml')
root = tree.getroot()

In [4]:
StationList = [] # MAC, [[min GPS],[max GPS]], [[first seen],[last seen]], device type, encryption, ESSID

In [5]:
def getDeviceType(bssid): # find vendor based off MAC address
    file = open('mac-vendors.txt', 'r', encoding="utf8")
    raw=[]
    mac_dict={}
    for line in file:
        raw.append(re.split(r'\t+', line.strip()))
    for pair in raw:
        mac_dict[pair[0]]= pair[1]
    return mac_dict.get(bssid[:8])

In [6]:
def haversine(lon1, lat1, lon2, lat2): # haversine formula for point distance
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    m = 6371* c/ 1.069
    return m

In [7]:
def appendArray(net, child): # parse xml and create array data
    if (child.tag == net) :
        bssid, essid = "", ""
        gps, time, encryption = [], [], []
        if (net=="wireless-network"): # network xml tag
            scan= 'BSSID'
        else:
            scan= 'client-mac'        # client xml tag
        for bssid_tag in child.findall(scan):
            bssid = bssid_tag.text
        for gps_tag in child.findall("gps-info"):
            gps= [[gps_tag[0].text, gps_tag[1].text],[gps_tag[4].text, gps_tag[5].text]]
        time = [child.attrib['first-time'],child.attrib['last-time']]
        for ssid_tag in child.findall('SSID'):
            for essid_tag in ssid_tag.findall('essid'):
                essid= essid_tag.text
            for info_tag in ssid_tag :
                if (info_tag.tag == "encryption"):
                    encryption.append(info_tag.text)
        StationList.append([bssid,gps,time,getDeviceType(bssid),encryption,essid]) 

In [8]:
for child in root:
    appendArray("wireless-network",child) #networks
    for subchild in child:
        appendArray("wireless-client", subchild) #clients
# lists all wireless devices
# pd.DataFrame(StationList, columns=["BSSID", "GPS","Dates","Manufacturer","Encryption","ESSID"])

### creep detecting starts here

In [13]:
creeps = []
for index in range(0,len(StationList)):
    distance = haversine(float(StationList[index][1][0][0]),float(StationList[index][1][0][1]),float(StationList[index][1][1][0]),float(StationList[index][1][1][1]))
    if (distance > .20): # .20 mile threshold
        creeps.append(StationList[index])
# creeps = list(set(creeps)) # remove duplicates , only works with BSSID
print (creeps)

[['00:0D:97:00:5B:33', [['34.190987', '-118.341560'], ['34.196323', '-118.340012']], ['Fri Oct 11 18:35:27 2019', 'Fri Oct 11 18:45:34 2019'], 'ABB Inc./Tropos', ['None'], 'BWP Free WiFi'], ['00:0D:97:00:5B:33', [['34.190987', '-118.341560'], ['34.196323', '-118.340012']], ['Fri Oct 11 18:35:27 2019', 'Fri Oct 11 18:45:34 2019'], 'ABB Inc./Tropos', [], ''], ['00:0D:97:09:B1:A4', [['34.190887', '-118.341789'], ['34.195740', '-118.340050']], ['Fri Oct 11 18:36:36 2019', 'Fri Oct 11 18:45:43 2019'], 'ABB Inc./Tropos', ['None'], None], ['00:0D:97:09:B1:A4', [['34.190887', '-118.341789'], ['34.195740', '-118.340050']], ['Fri Oct 11 18:36:36 2019', 'Fri Oct 11 18:45:43 2019'], 'ABB Inc./Tropos', ['None'], ''], ['10:DA:43:C7:32:ED', [['34.191067', '-118.341705'], ['34.193405', '-118.340103']], ['Fri Oct 11 18:37:01 2019', 'Fri Oct 11 18:43:06 2019'], 'NETGEAR', ['WPA+PSK', 'WPA+AES-CCM'], 'A-BEK-5G'], ['00:01:5C:A6:8A:46', [['34.191067', '-118.341705'], ['34.193405', '-118.340103']], ['Fri Oc

In [10]:
LDN_COORDINATES = (34.196136, -118.340164)
myMap = folium.Map(location=LDN_COORDINATES, zoom_start=60) 
display(myMap)